In [1]:
# NEURON simulator imports
from neuron import h, gui
from neuron.units import mV, ms, sec, um

# other essential packages
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

NEURON: unable to open font "*helvetica-medium-r-normal*--14*", using "fixed"


In [2]:
%matplotlib inline

In [3]:
# c compiler
!nrnivmodl

/home/hanna/Documents/studies/academies/2024-LASCON/project/synergisticDendrites
Mod files:

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
Successfully created x86_64/special


In [35]:
class Neuron():
    """
    Creates a NEURON cell instance with specific dendritic morphologies. Neurons can have passive or active
    somas and dendrites. They can be stimulated and return membrane potential recordings from pre- and post-
    synaptic stimulation sites and from the soma.
    
    Parameters
    ----------
    alpha (float): value between 0 and 1, indicating the pyramidal-likeness, as opposed to purkinje-likeness, 
                   of the dendritic morphology
    celsius (int): temperature in Celsius
    maxTreeLength (float): maximum length of dendritic tree in microns
    Vrest (float): resting potential in mV
    Cmem (float): membrane capacitance in uF cm^-2
    Rmem (float): membrane resistance in Ohm
    Ra (flat): axial resistance in Ohm
    geo (dict): geometry-defining default parameters of soma and dendritic sections
    mech (dict): mechanisms of dendrite, soma and axon Sections ('pas' for passive, or 'hh' for Hodgekin-Huxley)
    """
    
    def __init__(self,
                 alpha=0,
                 celsius=37,
                 Vrest=-70*mV,
                 Cmem=0.638856,
                 Rmem=120236.0,
                 Ra=141.949,
                 maxTreeLength=800*um,
                 geo={'soma': {'L':12.6157*um, 'diam': 12.6157*um, 'nseg': 5}, 
                      'dendrite': {'L': 10*um, 'diam': 1*um, 'nseg': 5}},
                 mech={'dendrite': 'pas', 'soma': 'hh', 'axon': 'pas'}):
        
        # reset h before creating a new neuron
        h('forall delete_section()')
        self.h = h
        h.celsius = celsius
        
        # set biophysical parameters (in soma)
        self.Vrest = Vrest
        self.Cmem = Cmem
        self.Rmem = Rmem
        self.Ra = Ra
        self.mech = mech

        # geometry defining parameters
        self.geo = geo
        self.alpha = alpha
        self.maxTreeLength = maxTreeLength

        # create soma, define its geometry and biophysics
        self.soma = self.createSection('soma')
        
        # create dendritic tree
        self.dendrites = []
        self.dendPaths = []
        self.growDendrites()

    def createSection(self, type):
        # create NEURON Section instance
        sec = h.Section(name=type, cell=self)
        
        # define Section geometry
        sec.L = self.geo[type]['L']
        sec.diam = self.geo[type]['diam']
        sec.nseg = self.geo[type]['nseg']
        
        # define biophysics
        sec.insert(self.mech[type])
        sec.Ra = self.Ra*(sec.diam/self.geo['soma']['diam'])
        sec.cm = self.Cmem*(self.geo['soma']['diam']/sec.diam)
        
        # set passive biophysical parameters
        if self.mech[type]=='pas':
            sec.e_pas = self.Vrest
            sec.g_pas = (1/self.Rmem)*(self.geo['soma']['diam']/sec.diam)
            
        # set hodgekin huxley biophysical parameters
        elif self.mech[type]=='hh':
            sec.gnabar_hh = 0.12  # Sodium conductance in S/cm2
            sec.gkbar_hh = 0.036  # Potassium conductance in S/cm2
            sec.gl_hh = 0.0003    # Leak conductance in S/cm2
            sec.el_hh = -54.3     # Reversal potential in mV

        return sec

    def addBranch(self, parent):
        """Creates a new dendrite section and connects it to the end of the parent. Returns dendrite section."""
        dend = self.createSection('dendrite')
        dend.connect(parent(1), 0)
        self.dendrites.append(dend)
        return dend

    def growDendrites(self, root=None):
        # if neuron has no dendrites yet, create one and connect to soma
        if not self.dendrites:
            root = self.addBranch(self.soma)
        elif not root:
            root = self.dendrite[-1]
            
        # initialize queue with new roots that can grow into branches
        roots = [[root]]   
        count = 0
        maxCount = 300

        while roots and count<maxCount:
            # pick a root and make it the current branch
            currentBranch = roots[-1]
            branchLength = len(currentBranch)*self.geo['dendrite']['L']
            del roots[-1]
            
            # extend branch until it has reached maximum length
            while branchLength<self.maxTreeLength:
                # add a new dendrite section
                dend = self.addBranch(currentBranch[-1])
                
                # check if bifurcation occurs 
                if np.random.rand()<self.getBranchProb(branchLength):
                    # connect sister branch to parent
                    sister = self.addBranch(currentBranch[-1])
                    # add sister branch to new roots
                    roots.append(currentBranch+[sister])
                    
                # add first branch to current branch list
                currentBranch.append(dend)
                
                # update branchLength
                branchLength += self.geo['dendrite']['L']
                count += 1

    def getBranchProb(self, dist2soma):
        # sample if cell acts like pyramidal or purkinje cell
        pyramidalLike = True if np.random.rand()<self.alpha else False
        # discretize probability distributions
        L = self.geo['dendrite']['L']
        d = dist2soma-L*0.5
        if pyramidalLike:
            mu = 5.26
            sigma = 1.55
            return (1/(d*sigma*np.sqrt(2*np.pi)))*np.exp(-((np.log(d)-mu)**2)/(2*sigma**2))*L
        else:
            return 0.5*(stats.gamma.pdf(d,a=2,scale=20)+stats.gamma.pdf(d,a=6.5,scale=30))*L

    def getDendriteIdx(self, dist2soma):
        """Returns indices of all dendrites with a certain distance (path length) from the soma."""
        pass

    def stimulate(self, n, T=1*sec, amp=1*mV, noise=0.2):
        pass

In [36]:
cell = Neuron()

In [37]:
# plot the neuron
shape_window = h.Shape()
shape_window.exec_menu('Show Diam')

0.0

In [17]:
# simulation parameters
t = 0*ms                   # simulation starts at t = 0 ms
dt = 0.01*ms               # integration time step, ms 